# RNN Building Blocks: injecting recursivity to supervised learner

In [3]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch import nn
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

import numpy as np
np.random.seed(0)

In [4]:
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch import nn

class mySeries(Dataset):
    """Series for RNN dataset."""

    def __init__(self,  length=1000, transform=None):
        self.transform = transform
        self.length = length

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        X = idx+2
        y = X+2

        return X,y

# mySeries dataset
trainset = mySeries()
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64)
